In [1]:
#Import libraries
import numpy as np
import sympy as sp
from typing import List
from sage.combinat.shuffle import ShuffleProduct
import itertools
from numpy import matrix 
from itertools import combinations as comb
import multiprocessing 
#from sage.parallel.multiprocessing_sage import Pool
from operator import add
import operator
from sys import getsizeof
import random
from numpy import array
from collections import Counter
import functools
import copy
import scipy 
import matplotlib.pyplot as plt
import networkx as nx
from multiprocessing import get_context
from sage.combinat.root_system.associahedron import Associahedra
from fractions import Fraction
from scipy.special import bernoulli, zeta 

# functions

def SortByIthTerms(ii, l): # ii starts from 1, from small to large
    r=[]

    if l!=[]: 
        t1=[]
        for i in l:
            t1.append(i[ii-1])

        t1=sorted(t1)

        r=[] 
        for i in [min(t1)..max(t1)]:
            for j in l:
                if j[ii-1]==i:
                    r.append(j)
    return r

def is_gap_2_every_where(L): # l = [[1, -5], [1, -3], [1, -1]]
    r=1
    l=SortByIthTerms(2,L)
    #print(l)
    for i in range(len(l)-1):
        if l[i+1][1]-l[i][1]!=2:
            r=0
            return r
    return r

def is_KR_module(l):
    r=1
    r1=[]
    for i in l:
        r1.append(i[0])
    r3=removeDuplicates(r1)
    if len(r3)>=2:
        r=0
        return r
    else:
        r2=is_gap_2_every_where(l)
        if r2==0:
            r=0
            return r
    return r

def is_Fundamental_module(l):
    if len(l)==1:
        r=1
    else:
        r=0
    
    return r

def define_fraction_field_with_variables_that_have_negative_indices(Y,k,n1,n2): # define ring with variables which have negative indices
    #k,n1,n2=4,-100,2, Y='x'
    indices = [(i,j) for i in range(1,k+1) for j in range(n1, n2)]
    R = PolynomialRing(QQ, [(Y+'_{}_{}').format(i, str(j).replace('-', 'm')) for (i,j) in indices])
    Y = dict(zip(indices, R.gens()))
    F=R.fraction_field()
    r=[F, Y]
    return r

def distance_of_indices_KR_module_half(i,typ,rank): # we use self-defined Cartan matrices, can change to Sage convention later
    r=1
    if typ=='G' and i==2:
        r=3
    elif typ=='B' and i!=rank:
        r=2
    elif typ=='C' and i==rank:
        r=2
    elif typ=='F' and (i==1 or i==2):  # this corresponds to self-defined Cartan matrix of type F4, can change to Sage convention later
        r=2
    return r

def KR_module_largest_s(i,s,k,typ,rank): # Y_{i,s}Y_{i,s-d}...
    d=distance_of_indices_KR_module_half(i,typ,rank)
    r=[]
    for j in [0..k-1]:
        r.append([i,s-2*j*d])
    return r



def InitialCluster_Yis_old(typ, rank, l): # C_l^{Dn}, l \ge 0, at the moment, typ='D'
    if typ=='A':
        n=rank
        p1=n*(l+1)
        
        vertices0=[]

        for i in [1..n]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-2, j+1,typ,rank)
                vertices0.append(sorted(t1))
                
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..n-2]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        for j in [0..l-1]:
            for i in [0..n-2]:
                t1=(i+1)*(l+1)+j+1
                t2=i*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1

        mutable_vertices=[]
        for i in [0..n-1]:
            for j in [0..l-1]:
                mutable_vertices.append(i*(l+1)+j)

        frozen_vertices=[]
        for i in [0..n-1]:
            frozen_vertices.append(i*(l+1)+l)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='B':
        n=rank
        p1=2*n*(l+1)
        
        vertices0=[]

        for i in [1..n-1]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, 2*n-2*i, j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in [n]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..2*l+1]:  # j=l are frozens
                t1=KR_module_largest_s(i, 1, j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in range(n-1,0,-1):
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, 2*i+2-2*n, j+1,typ,rank)
                vertices0.append(sorted(t1))
                
        #print(vertices0)
        
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver
        
        mat=Matrix(p1,p1)
        for i in [0..n-2]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        i=n-1
        for j in [0..2*l]:
            t1=i*(l+1)+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
            
        for i in [n+1..2*n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
                
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l]:
            t1=(n-2)*(l+1)+j
            t2=(n-1)*(l+1)+2*j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            t1=(n-1)*(l+1)+2*j
            t2=(n+1)*(l+1)+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for i in [n+1..2*n-2]:
            for j in [0..l]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l-1]:
            t1=(n-1)*(l+1)+2*j+1
            t2=(n-2)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l-1]:
            t1=(n+1)*(l+1)+j
            t2=(n-1)*(l+1)+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l-1]:
            for i in [n+1..2*n-2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        
        frozen_vertices=[]
        for i in [0..n-2]:
            frozen_vertices.append(i*(l+1)+l)
        i=n-1
        frozen_vertices.append((n+1)*(l+1)-1)
        for i in [n+1..2*n-1]:
            frozen_vertices.append(i*(l+1)+l)
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='C':
        n=rank
        p1=(n-1)*(l+1)+ceil(l/2)+ceil((l-1)/2)+2
        
        vertices0=[]

        for i in [1..n-1]:
            d=distance_of_indices_KR_module_half(i,typ,rank)
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, -i, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=n
        d=distance_of_indices_KR_module_half(i,typ,rank)
        for j in [0..ceil(l/2)]:  
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=n
        d=distance_of_indices_KR_module_half(i,typ,rank)
        for j in [0..ceil((l-1)/2)]:  
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
                
        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-2]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for i in [n-1]:
            for j in [0..ceil(l/2)-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for i in [n]:
            for j in [0..ceil((l-1)/2)-1]:
                t1=(n-1)*(l+1)+ceil(l/2)+1+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..ceil((l-1)/2)]:
            t1=(n-2)*(l+1)+2*j
            t2=(n-1)*(l+1)+ceil(l/2)+1+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..ceil(l/2)-1]:
            t1=(n-2)*(l+1)+2*j+1
            t2=(n-1)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..ceil(l/2)-1]:
            t1=(n-1)*(l+1)+j
            t2=(n-2)*(l+1)+2*j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..ceil((l-1)/2)-1]:
            t1=(n-1)*(l+1)+ceil(l/2)+j+1
            t2=(n-2)*(l+1)+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
  
        frozen_vertices=[]
        for i in [0..n-2]:
            frozen_vertices.append(i*(l+1)+l)
        frozen_vertices=frozen_vertices+[ (n-1)*(l+1)+ceil(l/2), (n-1)*(l+1)+ceil(l/2)+ceil((l-1)/2)+1 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices 
        
        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
    
    elif typ=='D':
        n=rank
        p1=n*(l+1)

        vertices0=[]
        
        for i in [1..n-1]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-2, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=n
        for j in [0..l]:   # j=l are frozens 
            t1=KR_module_largest_s(i, i-3, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j+1
                t2=i*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        i=n-3
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i+2)*(l+1)+j
            mat[t1,t2]=-1
            mat[t2,t1]=1

        i=n-3
        for j in [0..l-1]:
            t1=(i+2)*(l+1)+j+1
            t2=i*(l+1)+j
            mat[t1,t2]=-1
            mat[t2,t1]=1

        mutable_vertices=[]
        for i in [0..n-1]:
            for j in [0..l-1]:
                mutable_vertices.append(i*(l+1)+j)

        frozen_vertices=[]
        for i in [0..n-1]:
            frozen_vertices.append(i*(l+1)+l)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='E':  # 
        n=rank
        p1=n*(l+1)
        
        vertices0=[]
        xi=[-1,0,1,2]
        num=-1
        for i in [1,3,4,2]:
            num=num+1
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, xi[num], j+1,typ,rank)
                vertices0.append(sorted(t1))
        for i in [5..n]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-3, j+1,typ,rank)
                vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=4
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i-2)*(l+1)+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            for i in [5..n-1]:
                t1=i*(l+1)+j
                t2=(i-1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
                
        for j in [0..l-1]:
            for i in [0..2]+[4..n-2]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=2
        for j in [0..l-1]:
            t1=i*(l+1)+j
            t2=(i+2)*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1

        frozen_vertices=[]
        for i in [1..n]:
            frozen_vertices.append(i*l+i-1)
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='F':
        n=rank
        m1=ceil(l/2)
        m2=floor(l/2)
        p1=2*l+2*m1+2*m2+7
        
        vertices0=[]
        for i in [4,3]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, i-5, j+1,typ,rank)
                vertices0.append(sorted(t1))
        i=2
        for j in [0..m1]:  # j=l are frozens
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=1
        for j in [0..m1+1]:  # j=l are frozens
            t1=KR_module_largest_s(i, 1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=2
        for j in [0..m2]:  # j=l are frozens
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=1
        for j in [0..m2]:  # j=l are frozens
            t1=KR_module_largest_s(i, -5, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for j in [0..l-1]+[l+1..2*l]+[2*l+2..2*l+m1+1]+[2*l+m1+3..2*l+2*m1+3]+[2*l+2*m1+5..2*l+2*m1+m2+4]+[2*l+2*m1+m2+6..2*l+2*m1+2*m2+5]:
            t1=j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
            
        for j in [0..l]:
            t1=j
            t2=l+1+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2]:
            t1=l+2*j+1
            t2=2*l+2*m1+5+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2]:
            t1=2*l+2*m1+5+j
            t2=2*l+2*m1+m2+6+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [1..m1]:
            t1=l+2*j
            t2=2*l+2+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1]:
            t1=2*l+2+j
            t2=2*l+m1+4+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
            
        for j in [0..l-1]:
            t1=l+1+j
            t2=j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1-1]:
            t1=2*l+2+j
            t2=l+2*j+2
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m1]:
            t1=2*l+m1+3+j
            t2=2*l+2+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=2*l+2*m1+5+j
            t2=l+2*j+3
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=2*l+2*m1+m2+6+j
            t2=2*l+2*m1+6+j
            mat[t1,t2]=1
            mat[t2,t1]=-1
        
        
        frozen_vertices=[ l, 2*l+1, 2*l+m1+2, 2*l+2*m1+4, 2*l+2*m1+m2+5, 2*l+2*m1+2*m2+6 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    elif typ=='G':
        n=rank
        m1=ceil((l+1)/3)
        m2=ceil(l/3)
        m3=ceil((l-1)/3)
        p1=l+1+m1+m2+m3
        
        vertices0=[]
        i=1
        for j in [0..l]:  # j=l are frozens
            t1=KR_module_largest_s(i, 0, j+1,typ,rank)
            vertices0.append(sorted(t1))
        i=2
        for j in [0..m1-1]:   # 
            t1=KR_module_largest_s(i, -1, j+1,typ,rank)
            vertices0.append(sorted(t1))
        for j in [0..m2-1]:   # 
            t1=KR_module_largest_s(i, -3, j+1,typ,rank)
            vertices0.append(sorted(t1))
        for j in [0..m3-1]:   # 
            t1=KR_module_largest_s(i, -5, j+1,typ,rank)
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for j in [0..l-1]:
            t1=j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m1-2]:
            t1=l+1+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m2-2]:
            t1=l+1+m1+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m3-2]:
            t1=l+1+m1+m2+j
            mat[t1+1,t1]=1
            mat[t1,t1+1]=-1
        for j in [0..m1-1]:
            t1=j*3
            t2=l+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m2-1]:
            t1=j*3+1
            t2=l+m1+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..m3-1]:
            t1=j*3+2
            t2=l+m1+m2+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1
        for j in [0..l]:
            if j % 3 == 0 and j>=3: 
                t1=l+j/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
            elif j%3 == 1 and j>=4:
                t1=l+m1+(j-1)/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
            elif j%3 == 2 and j>=5:
                t1=l+m1+m2+(j-2)/3
                t2=j
                mat[t1,t2]=1
                mat[t2,t1]=-1
    
        frozen_vertices=[ l, l+m1, l+m1+m2, l+m1+m2+m3 ]
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    return r

def SetDifference(A,B): # A-B
    r=[d for d in A if d not in B]
    return r 

def ListAToN(a,n):
    r=list(range(a,n+1))
    return r

def merge_two_dicts(d1, d2):  
    d={}
    d2_keys_not_in_d1 = d2.keys() - d1.keys()
    d1_keys_not_in_d2 = d1.keys() - d2.keys()
    common_keys = d2.keys() & d1.keys()

    for i in common_keys:
        d[i]= removeDuplicates( d1[i]+d2[i] )
    for i in d1_keys_not_in_d2:
        d[i]=d1[i]
    for i in d2_keys_not_in_d1:
        d[i]=d2[i]
    return d

def ll_perms_compute_modules_Yis_compute_qcharacters(lli,typ,rank,max_degree,repeat,b6,fp1,l,variable2_ch,R,Y): #  l \ge 0, at the moment, typ='D'
# variable2_ch = [[[0, ch[[1, -1]], 0],
#   [0, ch[[1, -3], [1, -1]], 1],
#   [0, ch[[2, 0]], 2],
#   [0, ch[[2, -2], [2, 0]], 3],
#   [0, ch[[3, 1]], 4],
#   [0, ch[[3, -1], [3, 1]], 5],
#   [0, ch[[4, 1]], 6],
#   [0, ch[[4, -1], [4, 1]], 7]],
#  []]    

    #print(variable2_ch)
    
    IC=InitialCluster_Yis(typ,rank, l)
    mat1=IC[0]
    variable_ch=copy.deepcopy(variable2_ch)
    vertices1=[IC[1], variable_ch] 
    
    #print(vertices1)
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        #print(vertices1[1])
        
#         for i1 in range(mat1.nrows()):
#             for j1 in range(mat1.ncols()):
#                 if abs(mat1[i1][j1])>=2:
#                     print(MatrixTakeRows(mat1))
#                     print(str(vertices1)+'\n')
    
        if max_degree_of_Yis_in_monomial(vertices1[0][1])>max_degree:
            vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii=vertices1[0][1]  # highest weight
            ii2=vertices1[1][1] # qcharacter
            p1=repr(ii)
            p2=b6.get(p1)
        
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str([ii, ii2])+'\n')
                #b6[p1]=[] 
                b6[p1]=[[ii,ii2]] 
    return b6

def ll_perms_compute_modules_Yis_compute_qcharacters_old_initial_seed(lli,typ,rank,max_degree,repeat,b6,fp1,l,variable2_ch,R,Y): #  l \ge 0, at the moment, typ='D'
# variable2_ch = [[[0, ch[[1, -1]], 0],
#   [0, ch[[1, -3], [1, -1]], 1],
#   [0, ch[[2, 0]], 2],
#   [0, ch[[2, -2], [2, 0]], 3],
#   [0, ch[[3, 1]], 4],
#   [0, ch[[3, -1], [3, 1]], 5],
#   [0, ch[[4, 1]], 6],
#   [0, ch[[4, -1], [4, 1]], 7]],
#  []]    

    #print(variable2_ch)
    
    IC=InitialCluster_Yis_old(typ,rank, l)
    mat1=IC[0]
    variable_ch=copy.deepcopy(variable2_ch)
    vertices1=[IC[1], variable_ch] 
    
    #print(vertices1)
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
        
    for j in range(len(mutationSequence)): 
        vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
        mat1 = matrixMutation(mat1, mutationSequence[j]) 
        #print(vertices1[1])
        
#         for i1 in range(mat1.nrows()):
#             for j1 in range(mat1.ncols()):
#                 if abs(mat1[i1][j1])>=2:
#                     print(MatrixTakeRows(mat1))
#                     print(str(vertices1)+'\n')
    
        if max_degree_of_Yis_in_monomial(vertices1[0][1])>max_degree:
            vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
        else:
            ii=vertices1[0][1]  # highest weight
            ii2=vertices1[1][1] # qcharacter
            p1=repr(ii)
            p2=b6.get(p1)
        
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str([ii, ii2])+'\n')
                #b6[p1]=[] 
                b6[p1]=[[ii,ii2]] 
    return b6
               
def max_degree_of_Yis_in_monomial(l): # l is a list, l = [[2,-2],[2,-2],[1,-1]], max_degree of l is 2.
    r1=removeDuplicatesListOfLists(l)
    r=0
    for i in r1:
        t1=l.count(i)
        if r<t1:
            r=t1
    return r

def removeDuplicatesListOfLists(l): # very fast
    l.sort()
    r=list(l for l,_ in itertools.groupby(l))

    return r

# mutation of modules of quantum affine algebra of type Dn, C_l^g, l \ge 0, at the moment typ='D'
def computeEquationsForModulesYis_compute_qcharacter(variable0, mat, k, typ, rank, l, R, Y): # variable2=(variables on quiver, cluster variables obtained so far)
    
    #print(variable0)
    variable2=variable0[0]
    variable2_ch=variable0[1]
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[] 
    
    variable_ch=variable2_ch[0]
    clusterVariables_ch=variable2_ch[1] 
    newVariable_ch=1
    newVariable2_ch=1
    
    #print(variable2, variable2_ch)

    for i in range(size):
        if mat[i, k]>0:
            #print('i,variable[i]', i,variable[i])
            #print('power_of_monomial_Yis(variable[i][1], mat[i,k])', power_of_monomial_Yis(variable[i][1], mat[i,k]))
            newVariable=newVariable+power_of_monomial_Yis(variable[i][1], mat[i,k])
            #print('newVariable', newVariable)
            #print(variable_ch[i])
            for j in range(mat[i,k]):
                newVariable_ch=newVariable_ch*variable_ch[i][1]
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2=newVariable2 + power_of_monomial_Yis(variable[i][1], -mat[i,k])
            for j in range(-mat[i,k]):
                newVariable2_ch=newVariable2_ch*variable_ch[i][1]

    #print('newVariable_ch, newVariable2_ch, variable_ch[k][1]', newVariable_ch, newVariable2_ch, variable_ch[k][1])
    
    variable[k][0]=variable[k][0]+1
    variable_ch[k][0]=variable_ch[k][0]+1
    variable_ch[k][1]=simplify((newVariable_ch+newVariable2_ch)/variable_ch[k][1])
    
    t1=compare_weights_monomial_Yis(newVariable, newVariable2,typ,rank) 

    if t1==1: 
        variable[k][1]=sorted(monomial_Yis_division(newVariable, variable[k][1]))
    else:
        variable[k][1]=sorted(monomial_Yis_division(newVariable2, variable[k][1])) 
        
    clusterVariables=variable[k][1]
    clusterVariables_ch=variable_ch[k][1]
    #print('new cluster variable', clusterVariables)
    
    r=[[variable, clusterVariables], [variable_ch, clusterVariables_ch]]

    return r

def matrixMutation(mat,  k):  # mutate at k
    size=mat.nrows()
    r=Matrix(size,size)
    for i in range(size):
        for j in range(size):
            r[i,j]=mat[i,j]
    
    for i in range(size): 
        for j in range(size): 
            
            if k==i or k==j:
                r[i,j]=-mat[i, j]    
            else: 
                r[i, j] = mat[i, j]+1/2*(abs(mat[i,k])*mat[k,j]+mat[i,k]*abs(mat[k,j]))
     
    return r

def power_of_monomial_Yis(a,n):
    r=[]
    for i in range(n):
        r=r+a
    return r

def monomial_Yis_division(a,b):
    r=SetDifferenceListDifference2(a,b)
        
    return r

def SetDifferenceListDifference2(A,B): # A-B, can have duplicate elements
    r=[]
    r1=removeDuplicatesListOfLists(A)
    for i in r1:
        t1=A.count(i)-B.count(i)
        #print(t1)
        for j in range(1,t1+1):
            r.append(i)
            
    return r

def compare_weights_monomial_Yis(P1,P2,typ,rank): # 
    if P1!=[]:
        t1=weight_of_monomial_Yis(P1,rank)
    else:
        t1=[]
        for i in [1..rank]:
            t1.append(0)

    if P2!=[]:
        t2=weight_of_monomial_Yis(P2,rank)
    else:
        t2=[]
        for i in [1..rank]:
            t2.append(0)

    #print(t1,t2)

    r=compareWeights2(t1,t2,typ, rank)

    return r

def weight_of_monomial_Yis(a,rank): # a=[[1,-3],[2,0]]
    n=rank
    
    r=[]
    for i in [0..n-1]:
        r.append(0)
    
    for i in a:
        r[i[0]-1]=r[i[0]-1]+1
        
    return r 

def compareWeights2(a,b,typ,rank): # a,b are lists
    n=len(a)
    t1=Matrix(n,1)
    for i in range(n): 
        t1[i,0]=a[i] 
    t2=Matrix(n,1)
    for i in range(n): 
        t2[i,0]=b[i] 
    r=compareWeights(t1,t2,typ,rank)

    return r

def compareWeights(a, b, typ, rank): # 
    r=1                             # r=1 means a>=b          
    l=a-b
    c=CartanMatrixSelfDefined(typ, rank)
    for i in range(rank): 
        p=0
        for j in range(rank):
            t1=((transpose(c))^(-1))[j,i]
            p=p+l[j,0]*t1
        if p<0: 
            r=-1              # r=-1 means a is not >= b, it is possible that a<b or a,b are not comparable
            break 
            
    if r==-1:
        for i in range(rank):
            p=0
            for j in range(rank):
                t1=((transpose(c))^(-1))[j,i]
                p=p+l[j,0]*t1
            if p>0: 
                r=0
                break
    return r

def CartanMatrixSelfDefined(typ, rank):
    if typ=='A' or typ==1:
        r=CartanMatrix(['A',rank])
    elif typ=='B' or typ==2:
        r=CartanMatrix(['B',rank])
    elif typ=='C' or typ==3:
        r=CartanMatrix(['C',rank])
    elif typ=='D' or typ==4:
        r=CartanMatrix(['D',rank])
    elif typ=='E' or typ==5:
        r=CartanMatrix(['E',rank])
    elif typ=='F' or typ==6:
        r=CartanMatrix(['F',rank])
    elif typ=='G' or typ==7:
        r=CartanMatrix(['G',rank])
     
    return r 

def removeDuplicates(l):
    r=[]
    for i in l:
        if (i in r)==False:
            r.append(i)
    return r

def rho_Weyl(n): # [1,1,1,1,1] means omega_1+...
    r=[]
    for i in range(n):
        r.append(1)
    return r

def WeylGroupActionSiActionOnAlphaJ(ii,jj,typ,rank): # si(alphaj)=alphaj-C[j,i]alphai, < alpha_i, alpha_j^{\vee} > = C_{j,i}
    r=[]
    r1=[]
    n=rank
    for i in range(n):
        r.append(0)
        r1.append(0)
    r[jj-1]=1
    C=CartanMatrix([typ,rank])
    #print(C)
    t1=-C[jj-1,ii-1]
    r1[ii-1]=t1
    #print(r,r1)
    r=AddVlueOfListsMany([r,r1])
    return r

def WeylGroupActionSiActionOnCombinationOfAlphaJ(ii,la,typ,rank): # si(alphaj)=alphaj-C[j,i]alphai, < alpha_i, alpha_j^{\vee} > = C_{j,i}
    r=[] 
    n=rank
    for i in range(n):
        r.append(0)
    for j in range(len(la)):
        jj=j+1
        t1=WeylGroupActionSiActionOnAlphaJ(ii,jj,typ,rank) 
        t2=[la[j]*i for i in t1]
        r=AddVlueOfListsMany([r,t2])
    return r

def WeylGroupActionOnCombinationOfAlphaJ(w,la,typ,rank): # si(alphaj)=alphaj-C[j,i]alphai, < alpha_i, alpha_j^{\vee} > = C_{j,i}

    r=copy.deepcopy(la)
    if w==[]:
        r=copy.deepcopy(la)
    else:
        w1=ReverseList(w)
        for i in w1:
            r=WeylGroupActionSiActionOnCombinationOfAlphaJ(i,r, typ, rank)
    return r

def root_to_weight(la, typ,rank): # la = l1 alpha1 + .... = k1 omega1 + ...
    r1=[]
    r=[]
    n=rank
    weights=[]
    for i in range(n):
        r1.append(0)
        r.append(0)
    for j in range(n):
        r2=copy.deepcopy(r1)
        r2[j]=1 # r2=omega_j
        weights.append(r2)
    C=CartanMatrix([typ,rank])
    
    roots=[]
    for i in range(n):   # alpha_i = c1 omega_1 + ...
        t1=copy.deepcopy(r1)
        for j in range(n):
            t2=[C[i,j]*j1 for j1 in weights[j]]
            t1=AddVlueOfListsMany([t1,t2])
        roots.append(t1)
    for i in range(len(la)):
        c1=la[i]
        t1=[c1*j for j in roots[i]]
        r=AddVlueOfListsMany([r, t1])
    return r

def weight_to_root(om, typ,rank): # om = l1 omega_1+... = k1 alpha_1+...
    r1=[]
    r=[]
    n=rank
    roots=[]
    for i in range(n):
        r1.append(0)
        r.append(0)
    for j in range(n):
        r2=copy.deepcopy(r1)
        r2[j]=1 # r2=omega_j
        roots.append(r2)
    C=CartanMatrix([typ,rank])
    C_inv=C^(-1)
    weights=[]
    for i in range(n):   # omega_i = c1 alpha_1 + ...
        t1=copy.deepcopy(r1)
        for j in range(n):
            t2=[C_inv[i,j]*j1 for j1 in roots[j]]
            t1=AddVlueOfListsMany([t1,t2])
        weights.append(t1)
    for i in range(len(om)):
        c1=om[i]
        t1=[c1*j for j in weights[i]]
        r=AddVlueOfListsMany([r, t1])
    return r
    

def WeylGroupActionSiActionOnOmegaJ(ii,jj,typ,rank): # s_i(omega_j)
    n=rank
    C=CartanMatrix([typ,rank]) 
    om0=[]
    for i in range(n):
        om0.append(0)
    om0[jj-1]=1
    om=weight_to_root(om0, typ,rank)
    r1=WeylGroupActionOnCombinationOfAlphaJ([ii],om,typ,rank)
    r=root_to_weight(r1, typ, rank)
    return r

def WeylGroupActionSiAction(ii,la,typ,rank): # si(lambda),  lambda=[lambda1,lambda2,...]=lambda_1 omega_1 + lambda_2 omega_2 + ... 
    r=[]
    n=rank
    for i in range(n):
        r.append(0)
    for i in range(len(la)):
        c1=la[i]
        t1=WeylGroupActionSiActionOnOmegaJ(ii,i+1,typ,rank)
        t2=[c1*j for j in t1]
        r=AddVlueOfListsMany([r,t2])
    return r

def WeylGroupAction(w,la,typ,rank): # w =[1,2,1,3,2] is a product of si, w(la), la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
    r=copy.deepcopy(la)
    if w==[]:
        r=copy.deepcopy(la)
    else:
        w1=ReverseList(w)
        for i in w1:
            r=WeylGroupActionSiAction(i, r, typ, rank)
    return r


def Weyl_character_numerator(R,y,la,typ,rank): # la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
#     R=PolynomialRing(QQ,'y',200)
#     y=R.gens()  # y[i] = e^{omega_i}
    #W = WeylGroup([typ,rank], implementation="permutation")
    
    typ1=typ
    if typ=='B':
        typ1='C'
    elif typ=='C':
        typ1='B'
    
    W = WeylGroup([typ1,rank])
    n=rank
    rho=rho_Weyl(n) # omega_1+...+omega_n
    r=0
    for w1 in W:
        w=w1.reduced_word()
        #print('w',w)
        t0=AddVlueOfListsMany([rho,la])
        t1=WeylGroupAction(w,t0,typ1,rank)
        sgn=(-1)^(len(w))
        r1=1
        for i in range(len(t1)):
            r1=r1*y[i+1]^(t1[i])
        r=r+sgn*r1
        
    return r

def Weyl_character_denominator(R,y,typ,rank): # la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
#     R=PolynomialRing(QQ,'y',200)
#     y=R.gens()  # y[i] = e^{omega_i}
    #W = WeylGroup([typ,rank], implementation="permutation")
    
    typ1=typ
    if typ=='B':
        typ1='C'
    elif typ=='C':
        typ1='B'
    
    W = WeylGroup([typ1,rank])
    n=rank
    rho=rho_Weyl(n) # omega_1+...+omega_n
    r=0
    for w1 in W:
        w=w1.reduced_word()
        #print('w',w)
        t0=rho
        t1=WeylGroupAction(w,t0,typ1,rank)
        sgn=(-1)^(len(w))
        r1=1
        for i in range(len(t1)):
            r1=r1*y[i+1]^(t1[i])
        r=r+sgn*r1
        
    return r

def Weyl_character_formula(R,y,la,typ,rank): # la=lambda_1 omega_1 + lambda_2 omega_2 + ... 
    r1=Weyl_character_numerator(R,y,la,typ,rank)
    r2=Weyl_character_denominator(R,y,typ,rank)
    r=simplify(r1/r2)
        
    return r

def weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,m1,rank): # m1 = [1, y[1], y[1], y[2]], m1=y1y2y3
    r=[]
    n=rank
    for i in range(n):
        r.append(0)
    #print('m1', m1)
    for i in range(1,len(m1)):
        t1=y.index(m1[i])
        r[t1-1]=r[t1-1]+1
    return r

def highest_weight_term_of_character_of_module_of_Lie_algebra(R,y,p1,typ,rank): # p1=1/y[2] + y[2]/y[1] + y[1]
    r=[]
    r1=change_polynomial_to_list(R,numerator(p1))
    if len(r1)!=0:
        #print(r1[0])
        r=r1[0]
        wr=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r,rank)
        #print(r)
        for i in range(1,len(r1)):
            t1=r1[i]
            wt1=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r1[i],rank)
            t2=compareWeights2(wr,wt1,typ,rank)
            if t2==-1:
                r=t1
                wr=wt1
    r=ProductOfElementsInList(r)
    d1=denominator(p1)
    r=r/d1 

    return r

def monomial_to_weight(R,y,p1,rank): # 2*y[1]*y[2] = [2, 1, 1]
    r1=change_polynomial_to_list(R,numerator(p1))[0]
    r2=change_polynomial_to_list(R,denominator(p1))[0]
    #print(r1,r2)
    c1=r1[0]/r2[0]
    wr1=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r1,rank)
    wr2=weight_of_a_term_in_character_of_module_of_Lie_algebra(R,y,r2,rank)
    r3=[-j for j in wr2]
    r4=AddVlueOfListsMany([wr1,r3])
    r=[c1,r4]
    return r

def decompose_character_as_direct_sum_of_simple_module_of_Lie_algebra(R,y,p1,typ,rank):
    r1=copy.deepcopy(p1)
    wr1=highest_weight_term_of_character_of_module_of_Lie_algebra(R,y,p1,typ,rank)
    r=[]
    while r1!=0:
        #print('r1', r1)
        print(wr1)
        r2=monomial_to_weight(R,y,wr1,rank)
        #print('r2',r2)
        #print(Weyl_character_formula(R,y,r2[1],typ,rank))
        r3=simplify(r2[0]*Weyl_character_formula(R,y,r2[1],typ,rank))
        #print('r3',r3)
        r1=simplify(r1-r3)
        #print('r1',r1)
        wr1=highest_weight_term_of_character_of_module_of_Lie_algebra(R,y,r1,typ,rank)
        r.append(r2)
    return r

def change_polynomial_to_list(R, f):
    r=[[c] + sum([[SR(R.gen(i))]*e[i] for i in range(R.ngens()) if e[i] != 0], []) for (e,c) in R(f).dict().items()]
    return r

def ProductOfElementsInList(l):
    r=1
    for i in l:
        r=r*i
    return r

def AddVlueOfListsMany(l):
    r=[]
    if l!=[]:
        n=len(l[0])
        for i in range(n):
            r.append(0)
        for i in l:
            r=AddVlueOfTwoLists(r, i)
    return r

def AddVlueOfTwoLists(a,b):
    r=copy.deepcopy(a)
    for i in [0..len(a)-1]:
        r[i]=a[i]+b[i]
    return r

def ReverseList(l):
#     r=[]
#     for i in range(len(l)-1,-1,-1):
#         r=r+[l[i]]
    r=l[::-1]
    
    return r


def PositiveRootCorrespondingToIGammaI(a, i, tau, typ, rank):
    t1=CartanMatrixSelfDefined(typ, rank).inverse()
    di=diCartan(i,typ,rank)

    b1=0
    for k in range(t1.ncols()):
        b1=b1+t1[i-1,k]*a[k+1]
    b2=b1
    #print(b2)
    for k in range(di):
        b2=WeylGroupActionOnCombinationOfEJCoxeter2(tau, b2, a, typ, rank)
    
    #print(b1,b2)
    
    r=simplify(b1-b2)
        
    return r
    

def WeylGroupActionOnCombinationOfEJCoxeter2(w, la, alpha, typ, rank):
    r = la
    if w == []:
        r = la
    else:
        w1 = ReverseList(w)
        for i in w1:
            r = WeylGroupActionSiActionOnCombinationOfEJCoxeter2(i-1, r, alpha, typ, rank)
 
    return r 
    
def WeylGroupActionSiActionOnCombinationOfEJCoxeter2(i, la, alpha, typ, rank):
    t1 = ChangePolynomialToList([alpha], la)
    #print('la',la)
    #print(t1)
    t2 = 0
    for i1 in t1:
        j = alpha.index(i1[1])-1
        t2 = t2 + i1[0]*WeylGroupActionSiActionOnEJCoxeter2(i, alpha, j, typ, rank)
        #print(t2)
    r = simplify(t2)
    return r 
    
def WeylGroupActionSiActionOnEJCoxeter2(i, e, j, typ, rank): # 'star' is the automorphism of Dynkin diagram, folding
    
    #print('i', i)
    
    t1 = CartanMatrixSelfDefined(typ, rank)
    r = -t1[i,j]*e[i+1] + e[j+1]
    #print(i, r)
    return r 
    

def diCartan(i, typ, rank):
    n=rank
    r=1
    if typ=='B' and i<rank:
        r=2
    elif typ=='C' and i==rank:
        r=2
    elif typ=='F' and i<3:
        r=2
    elif typ=='G' and i==1:
        r=3
    
    return r
    
def CoefficientOftildeCij(i,j,n,typ,rank,N):
    #N=60
    qm=QuantumCartanMatrixSelfDefined(typ, rank)
    iqm=qm.inverse()
    t1=iqm[i-1,j-1]
    z=t1.parent().gen()
    
    t2=SR(t1).taylor(z,0,N)
    t3=t2.coefficients()
    
    r=0
    for k in t3:
        if k[1]==n:
            r=k[0]
            break
    return r


def DTildeOfInitialYitUsingCoxeterTransformationUseSForIxiNew(a, tau, i, s, L, typ, rank, N): # the case of I^{\xi}, remove epsilon_{i,j}
    r = 1
    for j in range(1, rank+1):
        g1 = PositiveRootCorrespondingToIGammaI(a, j, tau, typ, rank)

        s1=XiValue[j-1]

        c1=CoefficientOftildeCij(i,j,s1-s-1,typ,rank,N)
        c2=CoefficientOftildeCij(i,j,s1-s+1,typ,rank,N)

        p1=c1-c2

        t1 = g1^p1
        #print(type(t1))    
        r = r*t1
        #print(type(r))
        l = 0
        while s1-s+1 >= 0:
            s1=s1-2
            l=l+1
                
            #g1 = PositiveRootCorrespondingToIGammaI(a, j, tau, typ, rank)
            #for t2 in range(l):
                #g1 = WeylGroupActionOnCombinationOfEJCoxeter2(tau, g1, a, typ, rank)
                #print('l, s1, g1', l, s1, g1)
                
            g1 = WeylGroupActionOnCombinationOfEJCoxeter2(tau, g1, a, typ, rank)

            c1=CoefficientOftildeCij(i,j,s1-s-1,typ,rank,N)
            c2=CoefficientOftildeCij(i,j,s1-s+1,typ,rank,N)


            p1=c1-c2
            
            if p1!=0:
                t1 = g1^p1
                r = r*t1 
    #print(type(r))
    s1=numerator(r)
    s2=denominator(r)
    r=factor(s1)/factor(s2)
    #r=r.factor()
    #print(type(r))
    return r 


def Xi(i, type, rank):
    r=XiValue[i-1]
    
    if typ=='B' and i==1:
        r=r-2*(sigma-1)
    
    return r

def ConvertMonomialToList(l):
    c1=l.coefficients()[0]
    #print('c1',c1)
    l1=l/c1
    r1=[u for u in factor(l1)]
    r=[]
    #print('r1',r1)
    #print('l1',l1)
    for i in r1:
        #print('i',i)
        for j in range(i[1]):
            #print('i[0]',i[0])
            r.append(i[0])
    r2=c1
    r3=r
    for i in range(len(r)):
        if (r[i] in CC)==True:
            r2=r2*r[i]
            r3=removeAnElementInList(0,r3)
        else:
            break
    r=[r2]+r3
    return r

def ChangePolynomialToListHalf(c, l2):
    l = expand(l2)
    if l == 0:
        r = []
    elif len(l.coefficients()) == 1:
        #print('l',l)
        r = [ChangeMonomialToList(c, l)]
        #print('r',r)
    else: 
        r = []
        l1 = l.monomials()
        #print('l1',l1)
        l2=l.coefficients()
        #print(l2)
        for i in range(len(l1)):
            r.append(ChangeMonomialToList(c, l2[i]*l1[i]))
           
    return r 

def ChangePolynomialToList(c, l2):
    l = l2.denominator()  
    l1 = l2.numerator()
    t1 = ChangePolynomialToListHalf(c, l1)
    r = []
    for i in t1:
        r.append(UnionOfLists([i[0]/l], removeAnElementInList(0, i)))
    return r 

def ChangeMonomialToList(c, l):
    r1 = ConvertMonomialToList(l)
    if (r1[0] in CC) == True:
        r=r1
    else:
        r=[1]+r1
    return r 

def UnionOfLists(a,b):
    r=a+b
    return r

def UnionOfListsManyLists(l):
    r=[]
    for i in l:
        r=r+i
    return r

def removeAnElementInList(i, l):
 
    r=[]
    for j in range(len(l)):
        if (j!=i):
            r.append(l[j])
    
    return r

def removeAnElementInListList(i, l):
 
    r=[]
    for j in l:
        r.append(removeAnElementInList(i, j))
    
    return r


def QuantumCartanMatrixIJ(i,j, typ, rank):
    KK.<z> = FunctionField(QQ)
    di=diCartan(i,typ,rank)
    cij=CartanMatrixSelfDefined(typ,rank)[i-1,j-1]
    if i==j:
        r=z^di+z^(-di)
    elif i!=j:
        r=(z^(cij)-z^(-cij))/(z-1/z)
    
    r=simplify(r)
    return r

def QuantumCartanMatrixSelfDefined( typ, rank):
    KK.<z> = FunctionField(QQ)
    r=Matrix(KK, rank, rank)
    for i in range(1,rank+1):
        for j in range(1,rank+1):
            r[i-1,j-1]=QuantumCartanMatrixIJ(i,j,typ, rank)
    return r
 
def change_polynomial_to_list(R, f):
    r=[[c] + sum([[SR(R.gen(i))]*e[i] for i in range(R.ngens()) if e[i] != 0], []) for (e,c) in R(f).dict().items()]
    return r

# mutation of modules of quantum affine algebra of type Dn, C_l^g, l \ge 0, at the moment typ='D'
def computeEquationsForModulesYis(variable2, mat, k, typ, rank, l): # variable2=(variables on quiver, cluster variables obtained so far)
    
    variable=variable2[0]
    clusterVariables=variable2[1] 
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[]
    newVariableList=[]
    newVariable2List=[]

    for i in range(size):
        if mat[i, k]>0:
            #print('i,variable[i]', i,variable[i])
            #print('power_of_monomial_Yis(variable[i][1], mat[i,k])', power_of_monomial_Yis(variable[i][1], mat[i,k]))
            newVariable=newVariable+power_of_monomial_Yis(variable[i][1], mat[i,k])
            #print('newVariable', newVariable)
            for j in range(mat[i,k]):
                newVariableList.append(variable[i][1])
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2=newVariable2 + power_of_monomial_Yis(variable[i][1], -mat[i,k])
            for j in range(-mat[i,k]):
                newVariable2List.append(variable[i][1])
 
    variable[k][0]=variable[k][0]+1
    t1=compare_weights_monomial_Yis(newVariable, newVariable2,typ,rank)
    
    #print('t1,newVariable,newVariable2,variable[k][1]',[t1,newVariableList,newVariable2List,variable[k][1]]) 

    if t1==1: 
        variable[k][1]=sorted(monomial_Yis_division(newVariable, variable[k][1]))
    else:
        variable[k][1]=sorted(monomial_Yis_division(newVariable2, variable[k][1])) 
        
    clusterVariables=variable[k][1]
    #print('new cluster variable', clusterVariables)
    
    r=[variable, clusterVariables]

    return r

def ll_perms_compute_modules_Yis_compute_qcharacters_only_KR_modules(lli,typ,rank,max_degree,repeat,b6,fp1,l,variable2_ch,R,Y): #  l \ge 0, at the moment, typ='D'
# variable2_ch = [[[0, ch[[1, -1]], 0],
#   [0, ch[[1, -3], [1, -1]], 1],
#   [0, ch[[2, 0]], 2],
#   [0, ch[[2, -2], [2, 0]], 3],
#   [0, ch[[3, 1]], 4],
#   [0, ch[[3, -1], [3, 1]], 5],
#   [0, ch[[4, 1]], 6],
#   [0, ch[[4, -1], [4, 1]], 7]],
#  []]    

    #print(variable2_ch)
    
    IC=InitialCluster_Yis(typ,rank, l)
    mat1=IC[0]
    variable_ch=copy.deepcopy(variable2_ch)
    vertices1=[IC[1], variable_ch] 
    vertices_Yis=copy.deepcopy(IC[1])
    mat_Yis=copy.deepcopy(IC[0])
    
    #print(vertices1)
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
        
    for j in range(len(mutationSequence)): 
        vertices_Yis=computeEquationsForModulesYis(vertices_Yis, mat_Yis, mutationSequence[j], typ, rank, l)
        mat_Yis=matrixMutation(mat_Yis, mutationSequence[j]) 
        
        if max_degree_of_Yis_in_monomial(vertices_Yis[1])>max_degree or is_KR_module(vertices_Yis[1])==0:
            vertices_Yis=computeEquationsForModulesYis(vertices_Yis, mat_Yis, mutationSequence[j], typ, rank, l)
            mat_Yis=matrixMutation(mat_Yis, mutationSequence[j])
        else:
            vertices1 = computeEquationsForModulesYis_compute_qcharacter(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
            mat1 = matrixMutation(mat1, mutationSequence[j]) 
            
            ii=vertices1[0][1]  # highest weight
            ii2=vertices1[1][1] # qcharacter
            p1=repr(ii)
            p2=b6.get(p1)
        
            #print('p1',p1)
            
            if p2==None:
                #print('ii',ii)
                f1=open(fp1, 'a+')
                f1.write(str([ii, ii2])+'\n')
                #b6[p1]=[] 
                b6[p1]=[[ii,ii2]] 
    return b6

def randomMatrix(rank,n,minn,maxn): # 

    r=np.random.randint(minn,maxn,(rank, n))
    r=Matrix(r)
    
    return r

def random_list(n,minn,maxn):
    r=randomMatrix(1,n,minn,maxn)
    r=list(r[0])
    return r

# mutation of modules of quantum affine algebra of type Dn, C_l^g, l \ge 0, at the moment typ='D'
def computeEquationsForModulesYis_compute_qcharacter_division_use_multiply_inverse(variable0, mat, k, typ, rank, l, R, Y): # variable2=(variables on quiver, cluster variables obtained so far)
    
    #print(variable0)
    variable2=variable0[0]
    variable2_ch=variable0[1]
    variable=variable2[0]
    clusterVariables=variable2[1]
    size=mat.nrows() 
    newVariable=[]
    newVariable2=[] 
    
    variable_ch=variable2_ch[0]
    #print('variable_ch', variable_ch)
    clusterVariables_ch=variable2_ch[1]
    newVariable_ch=1
    newVariable2_ch=1
    rel1=[]
    rel2=[]
    rel3=[]
    
    #print(variable2, variable2_ch)

    for i in range(size):
        if mat[i, k]>0:
            #print('i,variable[i]', i,variable[i])
            #print('power_of_monomial_Yis(variable[i][1], mat[i,k])', power_of_monomial_Yis(variable[i][1], mat[i,k]))
            newVariable=newVariable+power_of_monomial_Yis(variable[i][1], mat[i,k])
            #print('newVariable', newVariable)
            #print(variable_ch[i])
            rel1.append(power_of_monomial_Yis(variable[i][1], mat[i,k]))
            #print(1)
            #print(variable[i][1])
            #print(variable_ch[i][1])
            for j in range(mat[i,k]):
                newVariable_ch=newVariable_ch*variable_ch[i][1]
 
    for i in range(size): 
        if mat[i, k]<0:
            newVariable2=newVariable2 + power_of_monomial_Yis(variable[i][1], -mat[i,k])
            rel2.append(power_of_monomial_Yis(variable[i][1], -mat[i,k]))
            #print(2)
            #print(variable[i][1])
            #print(variable_ch[i][1])
            for j in range(-mat[i,k]):
                newVariable2_ch=newVariable2_ch*variable_ch[i][1]

    #print('newVariable_ch, newVariable2_ch, variable_ch[k][1]', newVariable_ch, newVariable2_ch, variable_ch[k][1])
    #print('(newVariable_ch+newVariable2_ch)*variable_ch[k][1]^(-1)', factor(newVariable_ch+newVariable2_ch)*variable_ch[k][1]^(-1))
    
    #print(newVariable, newVariable_ch)
    #print(newVariable2, newVariable2_ch)
    
    variable[k][0]=variable[k][0]+1
    variable_ch[k][0]=variable_ch[k][0]+1
    s1=newVariable_ch+newVariable2_ch
    if s1!=0:
        s1=factor(s1)
        variable_ch[k][1]=s1*variable_ch[k][1]^(-1)
    else:
        variable_ch[k][1]=0
    
    temp=copy.deepcopy(variable[k][1])
    
    t1=compare_weights_monomial_Yis(newVariable, newVariable2,typ,rank) 
    
    if t1==1: 
        variable[k][1]=sorted(monomial_Yis_division(newVariable, variable[k][1]))
    else:
        variable[k][1]=sorted(monomial_Yis_division(newVariable2, variable[k][1])) 
        
    clusterVariables=variable[k][1]
    clusterVariables_ch=variable_ch[k][1]
    #print('new cluster variable', clusterVariables)
    
    rel3=[variable[k][1], temp]
    #print(rel3, rel1, rel2)
    
    r=[[variable, clusterVariables], [variable_ch, clusterVariables_ch]]

    return r

def ll_perms_compute_modules_Yis_compute_qcharacters_only_KR_modules_Dtilde(lli,typ,rank,max_degree,repeat,b6,fp1,l,variable2_ch,R,Y): #  l \ge 0, at the moment, typ='D'
# variable2_ch = [[[0, ch[[1, -1]], 0],
#   [0, ch[[1, -3], [1, -1]], 1],
#   [0, ch[[2, 0]], 2],
#   [0, ch[[2, -2], [2, 0]], 3],
#   [0, ch[[3, 1]], 4],
#   [0, ch[[3, -1], [3, 1]], 5],
#   [0, ch[[4, 1]], 6],
#   [0, ch[[4, -1], [4, 1]], 7]],
#  []]    

    #print(variable2_ch)
    
    IC=InitialCluster_Yis(typ,rank, l)
    mat1=IC[0]
    #print('1, variable2_ch', variable2_ch)
    variable_ch=variable2_ch
    vertices1=[IC[1], variable_ch] 
    vertices_Yis=copy.deepcopy(IC[1])
    mat_Yis=copy.deepcopy(IC[0])
    
    #print('2, variable_ch', variable_ch)
    
    #print(vertices1)
    
    ll=list(lli)
    #print(ll)
    
    mutationSequence=[]
    for j1 in [1..repeat]: 
        mutationSequence=mutationSequence+ll  
    #print(mutationSequence)
        
    for j in range(len(mutationSequence)): 
        #print(j,vertices1[1][0][mutationSequence[j]][1])
        if vertices1[1][0][mutationSequence[j]][1]!=0:
            vertices_Yis=computeEquationsForModulesYis(vertices_Yis, mat_Yis, mutationSequence[j], typ, rank, l)
            mat_Yis=matrixMutation(mat_Yis, mutationSequence[j]) 

            if max_degree_of_Yis_in_monomial(vertices_Yis[1])>max_degree or is_KR_module(vertices_Yis[1])==0:
                vertices_Yis=computeEquationsForModulesYis(vertices_Yis, mat_Yis, mutationSequence[j], typ, rank, l)
                mat_Yis=matrixMutation(mat_Yis, mutationSequence[j])
            else:
                vertices1 = computeEquationsForModulesYis_compute_qcharacter_division_use_multiply_inverse(vertices1, mat1, mutationSequence[j],typ,rank,l,R,Y)
                mat1 = matrixMutation(mat1, mutationSequence[j]) 

                ii=vertices1[0][1]  # highest weight
                ii2=vertices1[1][1] # qcharacter
                
                #print('ii2',simplify(ii2))
                
                p1=repr(ii)
                p2=b6.get(p1)

                #print('p1',p1)

                if p2==None:
                    #print('ii',ii)
                    f1=open(fp1, 'a+')
                    f1.write(str([ii, ii2])+'\n')
                    #b6[p1]=[] 
                    b6[p1]=[[ii,ii2]] 
    return b6


def InitialCluster_Yis(typ, rank, l): # C_l^{Dn}, l \ge 0, at the moment, typ='D'
    if typ!='D' and typ!='E':
        r=InitialCluster_Yis_old(typ,rank,l)
    elif typ=='E':
        n=rank
        p1=n*(l+1)
        
        vertices0=[]
        xi=[-1,-2,-3,-4]
        num=-1
        for i in [1,3,4,2]:
            num=num+1
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, xi[num], j+1,typ,rank)
                vertices0.append(sorted(t1))
                #print(sorted(t1))
        for i in [5..n]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, 1-i, j+1,typ,rank)
                vertices0.append(sorted(t1))
                #print(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
        i=4
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i-2)*(l+1)+j
            mat[t1,t2]=-1
            mat[t2,t1]=1
        for j in [0..l]:
            for i in [5..n-1]:
                t1=i*(l+1)+j
                t2=(i-1)*(l+1)+j
                mat[t1,t2]=-1
                mat[t2,t1]=1
                
        for j in [0..l-1]:
            for i in [0..2]+[4..n-2]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=2
        for j in [0..l-1]:
            t1=(i+2)*(l+1)+j
            t2=i*(l+1)+j+1
            
            mat[t1,t2]=1
            mat[t2,t1]=-1

        frozen_vertices=[]
        for i in [1..n]:
            frozen_vertices.append(i*l+i-1)
        
        mutable_vertices=SetDifference(ListAToN(0, p1-1), frozen_vertices)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
    elif typ=='D':
        n=rank
        p1=n*(l+1)

        vertices0=[]
        
        for i in [1..n-1]:
            for j in [0..l]:  # j=l are frozens
                t1=KR_module_largest_s(i, -i, j+1,typ,rank) # \xi(i)=-i
                vertices0.append(sorted(t1))
        i=n
        for j in [0..l]:   # j=l are frozens 
            t1=KR_module_largest_s(i, -n+1, j+1,typ,rank) # \xi(n)=-n+1
            vertices0.append(sorted(t1))

        verticesTableaux = [] # 
        for i in range(len(vertices0)):
            verticesTableaux.append([0, vertices0[i], i]) # 

        clusterVariables=[] 
        vertices1 = [verticesTableaux, clusterVariables] # vertices1[1] store cluster variables, vertices1[0] store variables on quiver

        mat=Matrix(p1,p1)
        for i in [0..n-1]:
            for j in [0..l-1]:
                t1=i*(l+1)+j
                mat[t1+1,t1]=1
                mat[t1,t1+1]=-1
        for j in [0..l]:
            for i in [0..n-3]:
                t1=i*(l+1)+j
                t2=(i+1)*(l+1)+j
                mat[t1,t2]=1
                mat[t2,t1]=-1
        for j in [0..l-1]:
            for i in [0..n-3]:
                t1=(i+1)*(l+1)+j
                t2=i*(l+1)+j+1
                mat[t1,t2]=1
                mat[t2,t1]=-1
        i=n-3
        for j in [0..l]:
            t1=i*(l+1)+j
            t2=(i+2)*(l+1)+j
            mat[t1,t2]=1
            mat[t2,t1]=-1

        i=n-3
        for j in [0..l-1]:
            t1=(i+2)*(l+1)+j
            t2=i*(l+1)+j+1
            mat[t1,t2]=1
            mat[t2,t1]=-1

        mutable_vertices=[]
        for i in [0..n-1]:
            for j in [0..l-1]:
                mutable_vertices.append(i*(l+1)+j)

        frozen_vertices=[]
        for i in [0..n-1]:
            frozen_vertices.append(i*(l+1)+l)

        initial_cluster_indices=mutable_vertices+frozen_vertices

        r=[mat, vertices1, mutable_vertices, initial_cluster_indices]
        
    return r

In [2]:
####################################################################
#compute Dtilde of KR modules directly using mutations 
#type E6, numerical
if __name__ == '__main__':
    #Define generation hyperparams
    typ='E'
    rank= 6   # 
    ell=8
     
    max_degree = 300 #...obtain only monomials with each Yis appears max_degree times  
    max_step = 1     #...this number controls the length of random mutation sequence, in order to obtain all cluster variables with number of columns less or equal to a fixed number, we need to put the number max_step sufficiently large
    repeat=1
    
    minn=3
    maxn=23
   
    fp1='ModulesUqghat'+'C'+str(ell)+str(typ)+str(rank)+'R'+str(max_degree)+'KR_modules_Dtilde_directly_numerical2.txt'
    
    L=[]
    tau = [1,3,4,2,5,6]
    XiValue=[-1,-4,-2,-3,-4,-5]
    n=rank
    N=4*ell+max(map(abs, XiValue))+rank

    n0=rank+1
#    n1,n2=-N,max(XiValue)
#    F,Y1=define_fraction_field_with_variables_that_have_negative_indices('Y1',n0,n1,n2) 
    Ra=PolynomialRing(QQ,'a',n0)
    Fa=Ra.fraction_field()
    a=Ra.gens()

    dict_Y={}
    for i in range(1,n+1):
        for q in range(rank+ell):
            s = Xi(i, typ, rank) - 2*q  
            #print(i,s)
            t1=DTildeOfInitialYitUsingCoxeterTransformationUseSForIxiNew(a, tau, i, s, L, typ, rank, N)
            dict_Y.update({str([i,s]):t1}) 
        
    IC=InitialCluster_Yis(typ,rank, ell)
    
#     a_random=random_list(rank+1,minn,maxn)
#     for i in [1..rank+1]:
#         a[i]=a_random[i-1]
    
    initial_Dtilde=[]
    for i in IC[1][0]:
        t1=1
        for j in i[1]:
            t1=t1*dict_Y.get(str([j[0],j[1]]))
        initial_Dtilde.append(t1)
    
#     Rx=PolynomialRing(QQ,'x',2*rank*ell)
#     x=Rx.gens() 
    
    F1 = open(fp1,'w+')
    F1.close() 
    
    #Run generation 
    
    ll0=IC[2] # mutable vertices
    #print(ll0)
    
#     ch0=[]
#     for i in [1..len(IC[1][0])]:
#         ch0.append(x[i])
#     #print(ch0)

    #ch0=initial_Dtilde
    
    t1=random_list(rank, minn, maxn)
    ch0=[]
    for j in initial_Dtilde:
        r1=SR(j)
        for i in range(len(t1)):
            r1=r1.subs({a[i+1]:t1[i]})
        ch0.append(r1)
    #print(ch0)
    
    va1=[]
    for i in range(len(ch0)):
        t1=[0,ch0[i],i]
        va1.append(t1)
    variable2_ch = [va1, []]   
    num,num1,sn,sn1=0,0,0,0
    b6={} 
    
    # to obtain all KR modules
    ll0=IC[2]
    t1=np.array_split(np.array(ll0),len(ll0)/ell)
    t10=[1,2,3,4,5,6]  
    t2=[]
    for i in t10:
        t2=t2+list(t1[i-1])
    lls = t2*(ell+1) 
    r1=ll_perms_compute_modules_Yis_compute_qcharacters_only_KR_modules_Dtilde(lls, typ, rank, max_degree, repeat, b6, fp1, ell, variable2_ch,Fa,a)

In [31]:
####################################################################
#compute Dtilde of KR modules directly using mutations 
#type E7, numerical
if __name__ == '__main__':
    #Define generation hyperparams
    typ='E'
    rank= 7   # 
    ell=9
     
    max_degree = 300 #...obtain only monomials with each Yis appears max_degree times  
    max_step = 1     #...this number controls the length of random mutation sequence, in order to obtain all cluster variables with number of columns less or equal to a fixed number, we need to put the number max_step sufficiently large
    repeat=1
    
    minn=3
    maxn=23
   
    fp1='ModulesUqghat'+'C'+str(ell)+str(typ)+str(rank)+'R'+str(max_degree)+'KR_modules_Dtilde_directly_numerical2.txt'
    
    L=[]
    tau = [1,3,4,2,5,6,7]
    XiValue=[-1,-4,-2,-3,-4,-5,-6]
    n=rank
    N=4*ell+max(map(abs, XiValue))+rank

    n0=rank+1
#    n1,n2=-N,max(XiValue)
#    F,Y1=define_fraction_field_with_variables_that_have_negative_indices('Y1',n0,n1,n2) 
    Ra=PolynomialRing(QQ,'a',n0)
    Fa=Ra.fraction_field()
    a=Ra.gens()

    dict_Y={}
    for i in range(1,n+1):
        for q in range(rank+ell):
            s = Xi(i, typ, rank) - 2*q  
            #print(i,s)
            t1=DTildeOfInitialYitUsingCoxeterTransformationUseSForIxiNew(a, tau, i, s, L, typ, rank, N)
            dict_Y.update({str([i,s]):t1}) 
        
    IC=InitialCluster_Yis(typ,rank, ell)
    
#     a_random=random_list(rank+1,minn,maxn)
#     for i in [1..rank+1]:
#         a[i]=a_random[i-1]
    
    initial_Dtilde=[]
    for i in IC[1][0]:
        t1=1
        for j in i[1]:
            t1=t1*dict_Y.get(str([j[0],j[1]]))
        initial_Dtilde.append(t1)
    
#     Rx=PolynomialRing(QQ,'x',2*rank*ell)
#     x=Rx.gens() 
    
    F1 = open(fp1,'w+')
    F1.close() 
    
    #Run generation 
    
    ll0=IC[2] # mutable vertices
    #print(ll0)
    
#     ch0=[]
#     for i in [1..len(IC[1][0])]:
#         ch0.append(x[i])
#     #print(ch0)

    #ch0=initial_Dtilde
    
    t1=random_list(rank, minn, maxn)
    ch0=[]
    for j in initial_Dtilde:
        r1=SR(j)
        for i in range(len(t1)):
            r1=r1.subs({a[i+1]:t1[i]})
        ch0.append(r1)
    #print(ch0)
    
    va1=[]
    for i in range(len(ch0)):
        t1=[0,ch0[i],i]
        va1.append(t1)
    variable2_ch = [va1, []]   
    num,num1,sn,sn1=0,0,0,0
    b6={} 
    
    # to obtain all KR modules
    ll0=IC[2]
    t1=np.array_split(np.array(ll0),len(ll0)/ell)
    t10=[1,2,3,4,5,6,7] 
    t2=[]
    for i in t10:
        t2=t2+list(t1[i-1])
    lls = t2*(ell+1) 
    r1=ll_perms_compute_modules_Yis_compute_qcharacters_only_KR_modules_Dtilde(lls, typ, rank, max_degree, repeat, b6, fp1, ell, variable2_ch,Fa,a)

In [ ]:
####################################################################
#compute Dtilde of KR modules directly using mutations 
#type E8, numerical
if __name__ == '__main__':
    #Define generation hyperparams
    typ='E'
    rank= 8   # 
    ell=10
     
    max_degree = 300 #...obtain only monomials with each Yis appears max_degree times  
    max_step = 1     #...this number controls the length of random mutation sequence, in order to obtain all cluster variables with number of columns less or equal to a fixed number, we need to put the number max_step sufficiently large
    repeat=1
    
    minn=3
    maxn=23
   
    fp1='ModulesUqghat'+'C'+str(ell)+str(typ)+str(rank)+'R'+str(max_degree)+'KR_modules_Dtilde_directly_numerical2.txt'
    
    L=[]
    tau = [1,3,4,2,5,6,7,8]
    XiValue=[-1,-4,-2,-3,-4,-5,-6,-7]
    n=rank
    N=4*ell+max(map(abs, XiValue))+rank

    n0=rank+1
#    n1,n2=-N,max(XiValue)
#    F,Y1=define_fraction_field_with_variables_that_have_negative_indices('Y1',n0,n1,n2) 
    Ra=PolynomialRing(QQ,'a',n0)
    Fa=Ra.fraction_field()
    a=Ra.gens()

    dict_Y={}
    for i in range(1,n+1):
        for q in range(rank+ell):
            s = Xi(i, typ, rank) - 2*q  
            #print(i,s)
            t1=DTildeOfInitialYitUsingCoxeterTransformationUseSForIxiNew(a, tau, i, s, L, typ, rank, N)
            dict_Y.update({str([i,s]):t1}) 
        
    IC=InitialCluster_Yis(typ,rank, ell)
    
#     a_random=random_list(rank+1,minn,maxn)
#     for i in [1..rank+1]:
#         a[i]=a_random[i-1]
    
    initial_Dtilde=[]
    for i in IC[1][0]:
        t1=1
        for j in i[1]:
            t1=t1*dict_Y.get(str([j[0],j[1]]))
        initial_Dtilde.append(t1)
    
#     Rx=PolynomialRing(QQ,'x',2*rank*ell)
#     x=Rx.gens() 
    
    F1 = open(fp1,'w+')
    F1.close() 
    
    #Run generation 
    
    ll0=IC[2] # mutable vertices
    #print(ll0)
    
#     ch0=[]
#     for i in [1..len(IC[1][0])]:
#         ch0.append(x[i])
#     #print(ch0)

    #ch0=initial_Dtilde
    
    t1=random_list(rank, minn, maxn)
    ch0=[]
    for j in initial_Dtilde:
        r1=SR(j)
        for i in range(len(t1)):
            r1=r1.subs({a[i+1]:t1[i]})
        ch0.append(r1)
    #print(ch0)
    
    va1=[]
    for i in range(len(ch0)):
        t1=[0,ch0[i],i]
        va1.append(t1)
    variable2_ch = [va1, []]   
    num,num1,sn,sn1=0,0,0,0
    b6={} 
    
    # to obtain all KR modules
    ll0=IC[2]
    t1=np.array_split(np.array(ll0),len(ll0)/ell)
    t10=[1,2,3,4,5,6,7,8] 
    t2=[]
    for i in t10:
        t2=t2+list(t1[i-1])
    lls = t2*(ell+1) 
    r1=ll_perms_compute_modules_Yis_compute_qcharacters_only_KR_modules_Dtilde(lls, typ, rank, max_degree, repeat, b6, fp1, ell, variable2_ch,Fa,a)